# Nobel prize analysis with API

The objective here is to get the nobel prize data via API and perform some analysis by converting data into dataframes.

In [1]:
# extension to help clean python code
%load_ext lab_black

## 1. Packages to install

In [2]:
# loading packages as per other notebook in repository
import json
import requests
import numpy as np
import pandas as pd
from pandas import json_normalize

## 2. Load data

In [3]:
# url to use for query
url = "http://api.nobelprize.org/v1/prize.json"

In [4]:
# get and deserialize data into dictionary
data = requests.get(url).json()
type(data)

dict

In [5]:
# save data in json format
with open("nobel_prize.json", "w") as out_file:
    json.dump(data, out_file)

In [6]:
# load data from nobel_prize.json if needed
# data = json.load(open('nobel_prize.json)

## 3. Explore data

In [7]:
# key values
data.keys()

dict_keys(['prizes'])

In [8]:
# explore the content of key 'prizes'
data["prizes"][:2]

[{'year': '2022',
  'category': 'chemistry',
  'laureates': [{'id': '1015',
    'firstname': 'Carolyn',
    'surname': 'Bertozzi',
    'motivation': '"for the development of click chemistry and bioorthogonal chemistry"',
    'share': '3'},
   {'id': '1016',
    'firstname': 'Morten',
    'surname': 'Meldal',
    'motivation': '"for the development of click chemistry and bioorthogonal chemistry"',
    'share': '3'},
   {'id': '743',
    'firstname': 'Barry',
    'surname': 'Sharpless',
    'motivation': '"for the development of click chemistry and bioorthogonal chemistry"',
    'share': '3'}]},
 {'year': '2022',
  'category': 'economics',
  'laureates': [{'id': '1021',
    'firstname': 'Ben',
    'surname': 'Bernanke',
    'motivation': '"for research on banks and financial crises"',
    'share': '3'},
   {'id': '1022',
    'firstname': 'Douglas',
    'surname': 'Diamond',
    'motivation': '"for research on banks and financial crises"',
    'share': '3'},
   {'id': '1023',
    'firstna

In [9]:
# check inner keys for single dictionary within key 'prizes'
data["prizes"][0].keys()

dict_keys(['year', 'category', 'laureates'])

## 4. Number of Nobel prizes given per category

Solution with dictionaries:

In [10]:
# 'prizes' key data
data_prizes = data["prizes"]

# empty lists
years = []
categories = []

# for loop
for item in data_prizes:
    years.append(item["year"])
    categories.append(item["category"])

# check output
print(years[:5])
print(categories[:5])

['2022', '2022', '2022', '2022', '2022']
['chemistry', 'economics', 'literature', 'peace', 'physics']


In [11]:
# find sorted unique elements of array
unique_categories = np.unique(categories)

In [12]:
# count nobel prizes per category using dictionary comprehension
category_counts = {k: categories.count(k) for k in unique_categories}

category_counts

{'chemistry': 122,
 'economics': 54,
 'literature': 122,
 'medicine': 122,
 'peace': 122,
 'physics': 122}

Solution with dataframes:

In [13]:
# normalize semi-structured JSON data into a flat table
df = json_normalize(data["prizes"])
df.head()

,year,category,laureates,overallMotivation
0,2022,chemistry,"[{'id': '1015', 'firstname': 'Carolyn', 'surna...",NaN
1,2022,economics,"[{'id': '1021', 'firstname': 'Ben', 'surname':...",NaN
2,2022,literature,"[{'id': '1017', 'firstname': 'Annie', 'surname...",NaN
3,2022,peace,"[{'id': '1018', 'firstname': 'Ales', 'surname'...",NaN
4,2022,physics,"[{'id': '1012', 'firstname': 'Alain', 'surname...",NaN


In [14]:
# groupby category, add count column, rename columns, sort descending, reset index
df.groupby(["category"]).agg({"category": "count"}).rename(
    columns={"category": "count"}
).sort_values("count", ascending=False).reset_index()

,category,count
0,chemistry,122
1,literature,122
2,medicine,122
3,peace,122
4,physics,122
5,economics,54


## 5. Flatten the data structure

The column 'laureates' contains another dictionary that we need to explode.

In [15]:
# check table format
df.head(1)

,year,category,laureates,overallMotivation
0,2022,chemistry,"[{'id': '1015', 'firstname': 'Carolyn', 'surna...",NaN


In [16]:
# table shape
df.shape

(664, 4)

In [17]:
# check for null values (NaN)
df.isna().sum()

year                   0
category               0
laureates             49
overallMotivation    606
dtype: int64

In [18]:
# drop 'laureates' null values before normalizing
df.laureates.isna().sum()

49

In [19]:
# if there were no null values in 'laureates' we could use
# df = json_normalize(data['prizes'],'laureates',['category','year'])

In [20]:
# drop rows for NaN laureates
df = df.dropna(subset=["laureates"])

# new table shape
df.shape

(615, 4)

In [21]:
# explode dictionary contained in 'laureates' column
df = df.explode("laureates").reset_index().rename(columns={"index": "prize_index"})

# check new table format
df.head()

,prize_index,year,category,laureates,overallMotivation
0,0,2022,chemistry,"{'id': '1015', 'firstname': 'Carolyn', 'surnam...",NaN
1,0,2022,chemistry,"{'id': '1016', 'firstname': 'Morten', 'surname...",NaN
2,0,2022,chemistry,"{'id': '743', 'firstname': 'Barry', 'surname':...",NaN
3,1,2022,economics,"{'id': '1021', 'firstname': 'Ben', 'surname': ...",NaN
4,1,2022,economics,"{'id': '1022', 'firstname': 'Douglas', 'surnam...",NaN


In [22]:
# number of rows increase as all 'laureates' are now unnested
df.shape

(989, 5)

In [23]:
# create new df of laureates to expand all information
df_laur = json_normalize(df.laureates)

# check new table format
df_laur.head()

,id,firstname,surname,motivation,share
0,1015,Carolyn,Bertozzi,"""for the development of click chemistry and bi...",3
1,1016,Morten,Meldal,"""for the development of click chemistry and bi...",3
2,743,Barry,Sharpless,"""for the development of click chemistry and bi...",3
3,1021,Ben,Bernanke,"""for research on banks and financial crises""",3
4,1022,Douglas,Diamond,"""for research on banks and financial crises""",3


In [24]:
# merge dataframes to get a final dataframe with all info needed
# axis = 1 -> concat columns
merged = pd.concat([df, df_laur], axis=1)

# new table
merged.head()

,prize_index,year,category,laureates,overallMotivation,id,firstname,surname,motivation,share
0,0,2022,chemistry,"{'id': '1015', 'firstname': 'Carolyn', 'surnam...",NaN,1015,Carolyn,Bertozzi,"""for the development of click chemistry and bi...",3
1,0,2022,chemistry,"{'id': '1016', 'firstname': 'Morten', 'surname...",NaN,1016,Morten,Meldal,"""for the development of click chemistry and bi...",3
2,0,2022,chemistry,"{'id': '743', 'firstname': 'Barry', 'surname':...",NaN,743,Barry,Sharpless,"""for the development of click chemistry and bi...",3
3,1,2022,economics,"{'id': '1021', 'firstname': 'Ben', 'surname': ...",NaN,1021,Ben,Bernanke,"""for research on banks and financial crises""",3
4,1,2022,economics,"{'id': '1022', 'firstname': 'Douglas', 'surnam...",NaN,1022,Douglas,Diamond,"""for research on banks and financial crises""",3


In [25]:
# drop unused columns
df = merged.drop(columns=["laureates", "overallMotivation"])

# new table
df.head()

,prize_index,year,category,id,firstname,surname,motivation,share
0,0,2022,chemistry,1015,Carolyn,Bertozzi,"""for the development of click chemistry and bi...",3
1,0,2022,chemistry,1016,Morten,Meldal,"""for the development of click chemistry and bi...",3
2,0,2022,chemistry,743,Barry,Sharpless,"""for the development of click chemistry and bi...",3
3,1,2022,economics,1021,Ben,Bernanke,"""for research on banks and financial crises""",3
4,1,2022,economics,1022,Douglas,Diamond,"""for research on banks and financial crises""",3


In [26]:
# reorder columns
df = df.reindex(
    columns=[
        "id",
        "firstname",
        "surname",
        "category",
        "year",
        "motivation",
        "share",
        "prize_index",
    ]
)

In [27]:
# dtype inspection
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           989 non-null    object
 1   firstname    988 non-null    object
 2   surname      956 non-null    object
 3   category     989 non-null    object
 4   year         989 non-null    object
 5   motivation   989 non-null    object
 6   share        989 non-null    object
 7   prize_index  989 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 61.9+ KB


In [28]:
# convert object to int for numbers
df["id"] = df["id"].astype(int)
df["year"] = df["year"].astype(int)
df["share"] = df["share"].astype(int)

# dtype check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           989 non-null    int64 
 1   firstname    988 non-null    object
 2   surname      956 non-null    object
 3   category     989 non-null    object
 4   year         989 non-null    int64 
 5   motivation   989 non-null    object
 6   share        989 non-null    int64 
 7   prize_index  989 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 61.9+ KB


In [29]:
# final table
df.head()

,id,firstname,surname,category,year,motivation,share,prize_index
0,1015,Carolyn,Bertozzi,chemistry,2022,"""for the development of click chemistry and bi...",3,0
1,1016,Morten,Meldal,chemistry,2022,"""for the development of click chemistry and bi...",3,0
2,743,Barry,Sharpless,chemistry,2022,"""for the development of click chemistry and bi...",3,0
3,1021,Ben,Bernanke,economics,2022,"""for research on banks and financial crises""",3,1
4,1022,Douglas,Diamond,economics,2022,"""for research on banks and financial crises""",3,1


Alternative faster method using flat_table:

In [30]:
# df = json_normalize(data['prizes'])

# import flat_table
# df_final = flat_table.normalize(df)

## 6. How many people called Michael got a Nobel prize?

In [31]:
# from previous df.info() we need to delete one missing firstname row
df["firstname"].isna().sum()

1

In [32]:
# drop missing firstname
df = df.dropna(subset=["firstname"])

# check deletion
df.shape

(988, 8)

In [33]:
# filter rows containing 'Michael' in column 'firstname'
df_michael = df[df["firstname"].str.contains("Michael")]
print(len(df_michael), "Michael's have received the Nobel Prize.")
df_michael

10 Michael's have received the Nobel Prize.


,id,firstname,surname,category,year,motivation,share,prize_index
37,986,Michael,Houghton,medicine,2020,"""for the discovery of Hepatitis C virus""",3,17
44,984,Michael,Kremer,economics,2019,"""for their experimental approach to alleviatin...",3,19
76,939,Michael,Rosbash,medicine,2017,"""for their discoveries of molecular mechanisms...",3,35
77,940,Michael W.,Young,medicine,2017,"""for their discoveries of molecular mechanisms...",3,35
87,930,J. Michael,Kosterlitz,physics,2016,"""for theoretical discoveries of topological ph...",4,40
114,890,Michael,Levitt,chemistry,2013,"""for the development of multiscale models for ...",3,54
259,745,A. Michael,Spence,economics,2001,"""for their analyses of markets with asymmetric...",3,127
352,279,Michael,Smith,chemistry,1993,"""for his fundamental contributions to the esta...",2,174
395,440,J. Michael,Bishop,medicine,1989,"""for their discovery of the cellular origin of...",2,203
435,432,Michael S.,Brown,medicine,1985,"""for their discoveries concerning the regulati...",2,227


## 7. Smallest relative share of Nobel prize ever given (same prize for maximum number of members).

In [34]:
# find maximum value in 'share' column
max_count = df["share"].max()
max_count

4

In [35]:
# answer
print("The smallest relative share ever given is {}%".format(1 / max_count * 100))

The smallest relative share ever given is 25.0%


## 8. Identify laureates who got more than a Noble prize.

The best approach here is to use the laureate id:

In [36]:
# pandas series value counts
multi_prizes_s = df["id"].value_counts()[df.id.value_counts() > 1]
multi_prizes_s

482    3
743    2
217    2
6      2
66     2
515    2
222    2
Name: id, dtype: int64

In [37]:
# pandas dataframe to get only leaureates ids that have won more than one prize
multi_prizes_df = df[df["id"].isin(multi_prizes_s.index)]
multi_prizes_df

,id,firstname,surname,category,year,motivation,share,prize_index
2,743,Barry,Sharpless,chemistry,2022,"""for the development of click chemistry and bi...",3,0
257,743,Barry,Sharpless,chemistry,2001,"""for his work on chirally catalysed oxidation ...",2,126
466,515,Office of the United Nations High Commissioner...,NaN,peace,1981,"""for promoting the fundamental rights of refug...",1,249
475,222,Frederick,Sanger,chemistry,1980,"""for their contributions concerning the determ...",4,252
568,66,John,Bardeen,physics,1972,"""for their jointly developed theory of superco...",3,304
639,482,International Committee of the Red Cross,NaN,peace,1963,"""for promoting the principles of the Geneva Co...",2,351
650,217,Linus,Pauling,peace,1962,"""for his fight against the nuclear arms race b...",1,356
674,222,Frederick,Sanger,chemistry,1958,"""for his work on the structure of proteins, es...",1,374
693,66,John,Bardeen,physics,1956,"""for their researches on semiconductors and th...",3,387
703,217,Linus,Pauling,chemistry,1954,"""for his research into the nature of the chemi...",1,394


In [38]:
# drop duplicates
df1 = multi_prizes_df[["id", "firstname", "surname"]].drop_duplicates().set_index("id")
df1

,firstname,surname
id,,
743,Barry,Sharpless
515,Office of the United Nations High Commissioner...,NaN
222,Frederick,Sanger
66,John,Bardeen
482,International Committee of the Red Cross,NaN
217,Linus,Pauling
6,Marie,Curie


In [39]:
# combine series and df for clearer table
result = df1.merge(
    multi_prizes_s.rename("count"), sort=True, left_index=True, right_index=True
).reset_index(drop=True)
result

,firstname,surname,count
0,Marie,Curie,2
1,John,Bardeen,2
2,Linus,Pauling,2
3,Frederick,Sanger,2
4,International Committee of the Red Cross,NaN,3
5,Office of the United Nations High Commissioner...,NaN,2
6,Barry,Sharpless,2


## 9. Which laureates from the ones that got multiple prizes got them in multiple categories?

In [40]:
# df laureates that got more than one prize
multi_prizes_df

,id,firstname,surname,category,year,motivation,share,prize_index
2,743,Barry,Sharpless,chemistry,2022,"""for the development of click chemistry and bi...",3,0
257,743,Barry,Sharpless,chemistry,2001,"""for his work on chirally catalysed oxidation ...",2,126
466,515,Office of the United Nations High Commissioner...,NaN,peace,1981,"""for promoting the fundamental rights of refug...",1,249
475,222,Frederick,Sanger,chemistry,1980,"""for their contributions concerning the determ...",4,252
568,66,John,Bardeen,physics,1972,"""for their jointly developed theory of superco...",3,304
639,482,International Committee of the Red Cross,NaN,peace,1963,"""for promoting the principles of the Geneva Co...",2,351
650,217,Linus,Pauling,peace,1962,"""for his fight against the nuclear arms race b...",1,356
674,222,Frederick,Sanger,chemistry,1958,"""for his work on the structure of proteins, es...",1,374
693,66,John,Bardeen,physics,1956,"""for their researches on semiconductors and th...",3,387
703,217,Linus,Pauling,chemistry,1954,"""for his research into the nature of the chemi...",1,394


In [41]:
# drop duplicates for ['id', 'category'] if any
multi_prizes_df.drop_duplicates(subset=["id", "category"])

,id,firstname,surname,category,year,motivation,share,prize_index
2,743,Barry,Sharpless,chemistry,2022,"""for the development of click chemistry and bi...",3,0
466,515,Office of the United Nations High Commissioner...,NaN,peace,1981,"""for promoting the fundamental rights of refug...",1,249
475,222,Frederick,Sanger,chemistry,1980,"""for their contributions concerning the determ...",4,252
568,66,John,Bardeen,physics,1972,"""for their jointly developed theory of superco...",3,304
639,482,International Committee of the Red Cross,NaN,peace,1963,"""for promoting the principles of the Geneva Co...",2,351
650,217,Linus,Pauling,peace,1962,"""for his fight against the nuclear arms race b...",1,356
703,217,Linus,Pauling,chemistry,1954,"""for his research into the nature of the chemi...",1,394
921,6,Marie,Curie,chemistry,1911,"""in recognition of her services to the advance...",1,609
974,6,Marie,Curie,physics,1903,"""in recognition of the extraordinary services ...",4,652


In [42]:
# non-unique names
multicat_df = multi_prizes_df.drop_duplicates(subset=["id", "category"])
multicat_s = multicat_df["id"].value_counts()
multicat_s

217    2
6      2
743    1
515    1
222    1
66     1
482    1
Name: id, dtype: int64

In [43]:
# get ids where unique categories > 1
more_categories = multicat_df[multicat_df["id"].isin(multicat_s[multicat_s > 1].index)]
more_categories

,id,firstname,surname,category,year,motivation,share,prize_index
650,217,Linus,Pauling,peace,1962,"""for his fight against the nuclear arms race b...",1,356
703,217,Linus,Pauling,chemistry,1954,"""for his research into the nature of the chemi...",1,394
921,6,Marie,Curie,chemistry,1911,"""in recognition of her services to the advance...",1,609
974,6,Marie,Curie,physics,1903,"""in recognition of the extraordinary services ...",4,652


In [44]:
# groupby 'id'
multicat_res_df = (
    more_categories.groupby("id")
    .agg(
        # take last element of the same firstname
        firstname=(
            "firstname",
            "last",
        ),
        # take the last element of the same surname
        surname=("surname", "last"),
        # join all categories awarded within the same column as str
        categories=("category", ", ".join),
    )
    .reset_index(drop=True)
)
multicat_res_df

,firstname,surname,categories
0,Marie,Curie,"chemistry, physics"
1,Linus,Pauling,"peace, chemistry"


In [45]:
multicat_res_df = (
    more_categories.groupby("id")
    .agg(
        {"firstname": "last", "surname": "last", "category": ", ".join},
    )
    .reset_index(drop=True)
)
multicat_res_df

,firstname,surname,category
0,Marie,Curie,"chemistry, physics"
1,Linus,Pauling,"peace, chemistry"


In [47]:
# alternative solution (without renaming)

# multicat_res_df = (
#    almost_there.groupby("id")
#    .agg({"firstname": "last", "surname": "last", "category": ", ".join},)
#    .reset_index(drop=True)
# )
# multicat_res_df